In [1]:
#import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pylab as pl
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import MiniBatchKMeans
from preprocess import *
from collections import Counter
import cv2
%load_ext autoreload
%autoreload 2

In [2]:
with open('fonts-dataset/font_names.txt') as file:
    lines = file.readlines()
    styles = [line.rstrip()[4:] for line in lines] # ignoring number at first of each style (first 4 chars)
print(styles)

['ScheherazadeNew', 'marhey', 'lemonada', 'ibm']


In [3]:
x=[]
y=[]
for i in range(0, len(styles)):
    for filename in sorted(glob.glob('fonts-dataset/'+str(i)+'/*.jpeg')):
        img = cv2.imread(filename)
     #   resized_img = cv2.resize(img, (200, 200))
        x.append(img)
        y.append(i)



In [4]:
x = x[0:5]
y =y[0:5]
x = np.asarray(x)
y = np.asarray(y)
print ("The size of the training set is: ", x.shape)
print ("The size of the labels of the training set is: ", y.shape)
print (y)

The size of the training set is:  (5, 1181, 1181, 3)
The size of the labels of the training set is:  (5,)
[0 0 0 0 0]


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
x_train_preprocessed = []
for img in x_train:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    x_train_preprocessed.extend(words)
print("x_train_preprocessed ", len(x_train_preprocessed))


x_train_preprocessed  295


In [6]:
from classifiers_manager import ClassifiersManager
classifiers_manager = ClassifiersManager()

def extract_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return descriptors

def prepare_data(images):
    features = []
    for image in images:
        descriptors = extract_sift_features(image)
        if descriptors  is not None:
            features.append(descriptors)
    return features

def create_codebook(features, num_clusters, batch_size):
    # Create a MiniBatchKMeans clustering object
    kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=batch_size, random_state=0)

    # Fit the clustering model to the SIFT features
    kmeans.fit(features)

    return kmeans

def compute_bovw_representation(features, codebook):
    num_clusters = codebook.n_clusters
    bovw_representation = []

    for image_features in features:
        if len(image_features) > 0:
            # Assign each feature to a cluster
            image_features=image_features.reshape(-1,128)
            cluster_assignments = codebook.predict(image_features)

            # Create a histogram of cluster frequencies
            histogram = np.bincount(cluster_assignments, minlength=num_clusters)

            # Normalize the histogram
            histogram = histogram / np.sum(histogram)

            bovw_representation.append(histogram)
        else:
            # Handle cases where no features were detected
            bovw_representation.append(np.zeros(num_clusters))

    return bovw_representation

In [7]:
x_train_preprocessed=prepare_data(x_train_preprocessed)
num_clusters=100
batch_size=1000


In [8]:
x_train_preprocessed = np.vstack(x_train_preprocessed)
print(type(x_train_preprocessed))
print(x_train_preprocessed.shape)
codebook = create_codebook(x_train_preprocessed, 100, 1000)
print(codebook)
train_bovw_features = compute_bovw_representation(x_train_preprocessed, codebook)


<class 'numpy.ndarray'>
(6441, 128)


AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
classifiers_manager.train_all_classifiers(train_bovw_features,y_train)

In [ ]:
x_test_preprocessed = []
for img in x_test:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    x_test_preprocessed.append(words)
print("x_test_preprocessed: ", len(x_test_preprocessed))

In [ ]:
x_test_preprocessed= prepare_data(x_test_preprocessed)
test_bovw_features = compute_bovw_representation(x_test_preprocessed, codebook)
classifiers_manager.test_all_classifiers(x_test_preprocessed,y_test,styles)
classifiers_manager.print_metrics()